In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.metrics import mean_squared_error
from pysindy.utils.odes import lorenz

# Ignore integration and solver convergence warnings
import warnings
from scipy.integrate.odepack import ODEintWarning
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=ODEintWarning)

import pysindy as ps

In [ ]:
x1 = [20,20,52,83,64,68,83,12,36,150,110,60,7,10,70,100,92,70,10,11,137,137,18,22,52,83,18,10,9,65]
x2 = [32,50,12,10,13,36,15,12,6,6,65,70,40,9,20,34,45,40,15,15,60,80,26,18,37,50,35,12,12,25]
dt = 1
t = np.arange(0, 29, dt)
X = np.c_[x1, x2]

plt.figure()
plt.plot(X)

In [ ]:
# Run SINDy
alpha = 100
threshold = 0.003
ensemble_optimizer = ps.STLSQ(threshold = threshold,alpha=alpha)
feature_names = ['x', 'y']
feature_library=ps.PolynomialLibrary(degree=3,library_ensemble=True, include_bias=False)
#functions = [lambda x : x, lambda x,y : x*y]
#feature_library = ps.CustomLibrary(library_functions=functions)
model = ps.SINDy(feature_names=feature_names,optimizer=ensemble_optimizer,feature_library=feature_library)
model.fit(X,t=dt,ensemble=True,n_models=500,n_subset=25,quiet=True,n_candidates_to_drop=0)
model.print()

t_horizon = 29
t_start = 0
x_sim = model.simulate(X[t_start,:],t[0:t_horizon])

# Visualization
fig, axs = plt.subplots(2)
fig.suptitle('SINDy model reconstruction')
axs[0].legend(['Lynxes','Hares'])
axs[0].set_title('Data')
axs[0].plot(X)
axs[0].grid()

axs[1].set_title('SINDy model')
axs[1].plot(t[t_start:t_horizon],x_sim)
#axs[1].ylim([0,140])
axs[1].grid()

plt.figure()
plt.plot(X)
plt.ylim([0,140])

print(X)

#Simulate 1-step prediction
for i in np.arange(1,29):
    x_sim = model.simulate(X[i,:],t[0:3])
    print(x_sim[1,:])
    plt.scatter(i+1,x_sim[1,0],c ="blue")
    plt.scatter(i+1,x_sim[1,1],c = "orange") 